<a href="https://colab.research.google.com/github/anamunoz01/PROYECTO-INTEGRADOR-III/blob/main/ProyectoI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**MODELO PREDICTIVO DE ATAQUES AL CORAZÓN CON REDES NEURONALES** 

**Proyecto Integrador III**

- Elena Gómez

- Ana Muñoz

- Inés Sánchez

- Miriam Toledo


## **Objetivo**

Vamos a predecir la posibilidad de sufrir un ataque al corazón dependiendo de diferentes características relacionadas con la salud del paciente.  

Para ello creamos un modelo de clasificación binaria, con el que obtendremos una salida que será 0 o 1.
- 0 significará que la probabilidad de sufrir un ataque al corazón es muy baja.
-1 será el caso contrario, es decir, que la probabilidad es alta.

La base de datos que utilizaremos la encontramos en kaggle.com, como Health care: Heart attack possibility. 

Las características que vamos a estudiar son las siguientes:
1. Edad
2. Sexo
3. Tipo de dolor de pecho (4 tipos)
4. Presión en sangre
5. Colesterol en mg/dl
6. Azúcar en sangre > 120 mg/dl
7. Resultado electrocardiograma (valores 0, 1, 2)
8. Frecuencia cardiaca máxima alcanzada
9. Angina de pecho (inducida por el ejercicio)
10. Depresión del ST inducida por el ejercicio en relación con el reposo
11. La pendiente del segmento ST máximo del ejercicio
12. Número de vasos mayores (0-3) coloreados por la fluoroscopia
13. thal: 0 = normal; 1 = defecto fijo; 2 = defecto reversible
14. objetivo: 0= menos posibilidades de infarto 1= más posibilidades de infarto

**Estudio y análisis de datos**

In [16]:
import keras
keras.__version__

'2.8.0'

In [17]:
import pandas as pd
df = pd.read_excel('dataset heart attack.xlsx')


In [18]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,57,0,0,140,241,0,1,123,1,0.2,1,0,3,0
299,45,1,3,110,264,0,1,132,0,1.2,1,0,3,0
300,68,1,0,144,193,1,1,141,0,3.4,1,2,3,0
301,57,1,0,130,131,0,1,115,1,1.2,1,1,3,0


In [19]:
train_data = df.sample(frac=0.8,random_state=0)
# El resto de datos los almacenamos en test_data:
test_data = df.drop(train_data.index)

In [20]:
train_data

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
225,70,1,0,145,174,0,1,125,1,2.6,0,0,3,0
152,64,1,3,170,227,0,0,155,0,0.6,1,0,3,1
228,59,1,3,170,288,0,0,159,0,0.2,1,0,3,0
201,60,1,0,125,258,0,0,141,1,2.8,1,1,3,0
52,62,1,2,130,231,0,1,146,0,1.8,1,3,3,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,60,0,2,102,318,0,1,160,0,0.0,2,1,2,1
285,46,1,0,140,311,0,1,120,1,1.8,1,2,3,0
11,48,0,2,130,275,0,1,139,0,0.2,2,0,2,1
119,46,0,0,138,243,0,0,152,1,0.0,1,0,2,1


Normalizamos

In [21]:
mean = train_data.mean(axis=0)
print(mean)
train_data -= mean
std = train_data.std(axis=0)
print(std)
train_data /= std
#print(train_data[1])

test_data -= mean
test_data /= std

age          54.128099
sex           0.685950
cp            0.962810
trestbps    131.396694
chol        247.181818
fbs           0.144628
restecg       0.528926
thalach     149.797521
exang         0.330579
oldpeak       0.991322
slope         1.413223
ca            0.685950
thal          2.322314
target        0.553719
dtype: float64
age          8.904992
sex          0.465098
cp           1.031989
trestbps    17.352331
chol        51.235037
fbs          0.352454
restecg      0.516522
thalach     22.935873
exang        0.471396
oldpeak      1.162148
slope        0.606595
ca           0.985649
thal         0.614012
target       0.498136
dtype: float64


In [22]:
train_labels = train_data.pop('target')
test_labels = test_data.pop('target')

Creamos la red

In [23]:
import tensorflow as tf
from keras import models
from keras import layers

def build_model():
    # Porque necesitaremos instanciar el mismo modelo 
    # múltiples veces, usamos una función para construirlo
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu',
                           input_shape=(13,)))
    model.add(layers.Dense(16, activation='relu'))
    model.add(layers.Dense(1, activation = 'sigmoid'))
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy']) 
    return model

In [24]:
import numpy as np

k = 3
num_val_samples = len(train_data) // k
print("num_val_samples", num_val_samples)
num_epochs = 50
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Preparamos la validación de los datos: datos de la partición #k
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    val_labels = train_labels[i * num_val_samples: (i + 1) * num_val_samples]

    # Preparamos los datos de entrenamiento: Datos de las demás particiones
    partial_train_data = np.concatenate(
        [train_data[:i * num_val_samples],
         train_data[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_labels = np.concatenate(
        [train_labels[:i * num_val_samples],
         train_labels[(i + 1) * num_val_samples:]],
        axis=0)

    # Construcción del modelo Keras (ya compilado)
    model = build_model()
    # Entrenando el modelo (en modo silencioso, verbose=0)
    model.fit(partial_train_data, partial_train_labels,
              epochs=num_epochs, batch_size=1, verbose=0)
    # Evalua el modelo sobre los datos de validación
    x = model.evaluate(val_data, val_labels, verbose=0)
    all_scores.append(x)

num_val_samples 80
processing fold # 0
processing fold # 1
processing fold # 2


In [25]:
all_scores

[[-23377.23046875, 0.0], [-18673.734375, 0.0], [-27468.47265625, 0.0]]